# ST IT Cloud - Data and Analytics Test LV.3

Esse teste deve avaliar a qualidade técnica na manipulacão de dados.

## Passo a passo

- Disponibilizamos aqui 1 case para ser solucionado, leia os enunciados dos problemas, desenvolva os programas, utilizando a **stack definida durante o processo seletivo**, para entregar os dados de acordo com os requisitos descritos abaixo.


**Faz parte dos critérios de avaliacão a pontualidade da entrega. Implemente até onde for possível dentro do prazo acordado.**

**Os dados de pessoas foram gerados de forma aleatória, utilizando a biblioteca FakerJS, FakerJS-BR e Faker**

LEMBRE-SE: A entrega deve conter TODOS os passos para o avaliador executar o programa (keep it simple).


# TESTE PRÁTICO

**Problema**: Você está recebendo o arquivo 'dados_cadastrais_fake.csv' que contem dados cadastrais de clientes, mas para que análises ou relatórios sejam feitos é necessário limpar e normalizar os dados. Além disso, existe uma coluna com o número de cpf e outra com cnpj, você precisará padronizar deixando apenas dígitos em formato string (sem caracteres especiais), implementar uma forma de verificar se tais documentos são válidos sendo que a informação deve se adicionada ao dataframe em outras duas novas colunas.

Após a normalização, gere reports que respondam as seguintes perguntas:
- Quantos clientes temos nessa base?
- Qual a média de idade dos clientes?
- Quantos clientes nessa base pertencem a cada estado?
- Quantos CPFs válidos e inválidos foram encontrados?
- Quantos CNPJs válidos e inválidos foram encontrados?

Ao final gere um arquivo no formato csv e um outro arquivo no formato parquet chamado (problema1_normalizado), eles serão destinados para pessoas distintas.

In [60]:
#Como utilizei o Google Colab para realizar esse desafio, tive que instalar algumas dependências:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 499, in run
    conflicts = self._determine_conflicts(to

In [132]:
from pyspark.sql import SparkSession
from pyspark.sql.types import BooleanType, StringType
from pyspark.sql.functions import initcap, regexp_replace, ltrim, rtrim, col, udf

In [169]:
spark = SparkSession.builder.appName("problema1").getOrCreate()
df = spark.read.option("header", True).option("delimiter", ";").option("inferSchema", True).csv("dados_cadastrais_fake.csv")
df.show()

+------------------+-----+-------------------+----------------+--------------+------------------+
|             nomes|idade|             cidade|          estado|           cpf|              cnpj|
+------------------+-----+-------------------+----------------+--------------+------------------+
|    Dennis Daniels|   31|         ACRELÂNDIA|              AC|   97566536800|    06589184909526|
|       Leah Becker|   42|        ÁGUA BRANCA|              AL|425.263.807-07|25.673.336/2350-20|
|        Sally Ford|   18|           ALVARÃES|              AM|   34647754103|    26543101702989|
|    Colleen Duncan|   21|     SERRA DO NAVIO|              AP|252.531.560-03|19.062.080/5100-98|
|   Jeff Stephenson|   73|             ABAÍRA|              BA|   49668886542|    97794530015384|
|     Sydney Curtis|   85|            ABAIARA|              CE|506.202.907-49|29.476.298/0856-78|
|    Kelly Matthews|   44|           Brasília|distrito federal|   39154836808|    24709301957761|
|         Juan Ruiz|

In [170]:
#iniciando a identificação de possiveis sujeiras nos dados.
#identificando sujeira nos nomes
df.filter(
    ~col("nomes").rlike("^([A-Z]([-']?[a-z]+))([ ]([A-Z]([-']?[a-z]+)[.]?)?([A-Z]+)?)*$")
).show(10, False)

+----------------------+-----+-------------------+------+--------------+------------------+
|nomes                 |idade|cidade             |estado|cpf           |cnpj              |
+----------------------+-----+-------------------+------+--------------+------------------+
|Mr. Andrew Chapman MD |55   |ABDON BATISTA      |SC    |645.976.852-86|26.848.890/5356-10|
|Mrs. Yesenia Barrett  |88   |ÁGUA BRANCA        |AL    |746.102.105-25|50.800.865/6979-77|
|Dr. Jennifer Williams |36   |SERRA DO NAVIO     |AP    |443.310.261-08|48.607.469/0454-82|
|Mr. Jesse Fletcher    |21   |ABADIA DOS DOURADOS|MG    |812.821.490-08|48.249.914/7290-01|
|Mr. Derrick Walker III|67   |SERRA DO NAVIO     |AP    |867.098.157-22|72.983.983/2334-06|
|Mr. Charles Schwartz  |47   |ÁGUA BRANCA        |PB    |01473369290   |78060778010570    |
|Mrs. Suzanne Peters   |19   |ACRELÂNDIA         |AC    |81433726181   |29592536264459    |
|Mrs. Angela Hernandez |34   |AFONSO CLÁUDIO     |ES    |967.654.794-81|78.156.6

In [174]:
#identificando a sujeito na idade
df.filter(
    (col("idade").cast("int").isNull()) | (col("idade").cast("int") <= 0) | (col("idade").cast("int") > 100)
).show(10, False)

+-----+-----+------+------+---+----+
|nomes|idade|cidade|estado|cpf|cnpj|
+-----+-----+------+------+---+----+
+-----+-----+------+------+---+----+



In [175]:
#identificando a sujeira na cidade
df.filter(
    ~col("cidade").rlike("^[\\p{L}]+([ ]([A-Z]['])?[\\p{L}]+)*$")
).show(10, False)

+-----+-----+------+------+---+----+
|nomes|idade|cidade|estado|cpf|cnpj|
+-----+-----+------+------+---+----+
+-----+-----+------+------+---+----+



In [176]:
#identificando a sujeira no estado
states_list = ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]
df.filter(
    ~col("estado").isin(states_list)
).show(10, False)

+-----------------+-----+----------+----------------+--------------+------------------+
|nomes            |idade|cidade    |estado          |cpf           |cnpj              |
+-----------------+-----+----------+----------------+--------------+------------------+
|Kelly Matthews   |44   |Brasília  |distrito federal|39154836808   |24709301957761    |
|Michelle Sullivan|45   |Brasília  |distrito federal|603.384.507-26|18.355.115/6493-65|
|Cindy Brown      |28   |Brasília  |distrito federal|11561438537   |89158131394480    |
|Patricia Meyer MD|31   |Brasília  |distrito federal|432.435.813-38|68.879.919/2769-85|
|Christopher Young|38   |Brasília  |distrito federal|24594975984   |71848458367870    |
|Elizabeth Miller |79   |Brasília  |distrito federal|723.509.900-36|41.178.451/8696-87|
|Barbara Leon MD  |73   |ADAMANTINA|sao  paulo      |05086323231   |98992168836002    |
|Stephen Torres   |48   |ADAMANTINA|sao  paulo      |922.545.994-70|51.006.075/0099-24|
|Kelli Poole      |22   |ADAMANT

In [177]:
#identificando a sujeira no cpf

df = df.withColumn("cpf", regexp_replace(col("cpf"), "[^0-9]", ""))

df.filter(
    ~col("cpf").rlike("^[\\d]{11}$")
).show(10, False)

+-----+-----+------+------+---+----+
|nomes|idade|cidade|estado|cpf|cnpj|
+-----+-----+------+------+---+----+
+-----+-----+------+------+---+----+



In [178]:
#identificando a sujeira no cnpj

df = df.withColumn("cnpj", regexp_replace(col("cnpj"), "[^0-9]", ""))

df.filter(
    ~col("cnpj").rlike("^[\\d]{14}$")
).show(10, False)

+-----+-----+------+------+---+----+
|nomes|idade|cidade|estado|cpf|cnpj|
+-----+-----+------+------+---+----+
+-----+-----+------+------+---+----+



In [179]:
#Iniciando a limpeza e normalização dos dados
# Normalizando nomes

df = df.withColumn("nomes", initcap(col("nomes")))
df = df.withColumn("nomes", regexp_replace(col("nomes"), "^[A-Z][a-z]*[.][ ]", ""))
df = df.withColumn("nomes", regexp_replace(col("nomes"), "^Miss[ ]", ""))

In [180]:
#Normalizando cidade

df = df.withColumn("cidade", initcap(col("cidade")))

In [181]:
#Limpando e normalizando estados

df = df.withColumn("estado", ltrim(rtrim(col("estado"))))
df = df.withColumn("estado", regexp_replace(col("estado"), "distrito federal", "DF"))
df = df.withColumn("estado", regexp_replace(col("estado"), "sao  paulo", "SP"))
df = df.withColumn("estado", regexp_replace(col("estado"), "são  paulo", "SP"))
df = df.withColumn("estado", regexp_replace(col("estado"), "rio de  janeiro", "RJ"))
df = df.withColumn("estado", regexp_replace(col("estado"), "MINAS GERAI", "MG"))
df = df.withColumn("estado", regexp_replace(col("estado"), "MGs", "MG"))

In [182]:
#validando o cpf e adicionando a nova coluna
def cpf_is_valid(cpf):
    cpf_numbers = [int(char) for char in cpf]
    
    # Validates CPF all same digits
    if cpf_numbers == cpf_numbers[::-1]:
        return "Não Valido"

    #  Validates last two digits
    for i in range(9, 11):
        value = sum((cpf_numbers[num] * ((i+1) - num) for num in range(0, i)))
        digit = ((value * 10) % 11) % 10
        if digit != cpf_numbers[i]:
            return "Não Valido"
    return "Valido"

valid_cpf = udf(lambda x: cpf_is_valid(x), StringType())
df = df.withColumn("cpf_status", valid_cpf(df.cpf))   

In [183]:
#validando o cnpj e adicionando a nova coluna

def cnpj_is_valid(cnpj):
    cnpj_numbers = [int(char) for char in cnpj]
    
    if len(cnpj_numbers) != 14:
        return "Não Valido"
    
    # Validates CNPJ all same digits
    if cnpj_numbers == cnpj_numbers[::-1]:
        return "Não Valido"

    #  Validates last two digits
    prod = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    beginning =  cnpj_numbers[:12]
    while len(beginning) < 14:
        r = sum([x*y for (x, y) in zip(beginning, prod)]) % 11
        if r > 1:
            f = 11 - r
        else:
            f = 0
        beginning.append(f)
        prod.insert(0, 6)
    
    if beginning != cnpj_numbers:
        return "Não Valido"
        
    return "Valido"

valid_cnpj = udf(lambda x: cnpj_is_valid(x), StringType())
df = df.withColumn("cnpj_status", valid_cnpj(df.cnpj))

In [144]:
df.show()

+------------------+-----+-------------------+------+-----------+--------------+----------+-----------+
|             nomes|idade|             cidade|estado|        cpf|          cnpj|cpf_status|cnpj_status|
+------------------+-----+-------------------+------+-----------+--------------+----------+-----------+
|    Dennis Daniels|   31|         ACRELÂNDIA|    AC|97566536800|06589184909526|    Valido|     Valido|
|       Leah Becker|   42|        ÁGUA BRANCA|    AL|42526380707|25673336235020|    Valido|     Valido|
|        Sally Ford|   18|           ALVARÃES|    AM|34647754103|26543101702989|    Valido|     Valido|
|    Colleen Duncan|   21|     SERRA DO NAVIO|    AP|25253156003|19062080510098|    Valido|     Valido|
|   Jeff Stephenson|   73|             ABAÍRA|    BA|49668886542|97794530015384|    Valido|     Valido|
|     Sydney Curtis|   85|            ABAIARA|    CE|50620290749|29476298085678|    Valido|     Valido|
|    Kelly Matthews|   44|           Brasília|    DF|39154836808

Respondendo as perguntas:

In [184]:
#Quantos clientes temos nessa base?

df.select("cpf").distinct().count()

10000

In [185]:
#Qual a média de idade dos clientes?

df.agg({"idade": "avg"}).show()

+----------+
|avg(idade)|
+----------+
|   53.7831|
+----------+



In [186]:
#Quantos clientes nessa base pertencem a cada estado?

df.groupBy("estado").count().show(30)

+------+-----+
|estado|count|
+------+-----+
|    SC|  370|
|    RO|  370|
|    PI|  370|
|    AM|  371|
|    RR|  370|
|    GO|  371|
|    TO|  370|
|    MT|  370|
|    SP|  370|
|    ES|  371|
|    PB|  370|
|    RS|  370|
|    MS|  370|
|    AL|  371|
|    MG|  370|
|    PA|  370|
|    BA|  371|
|    SE|  370|
|    PE|  370|
|    CE|  371|
|    RN|  370|
|    RJ|  370|
|    MA|  371|
|    AC|  371|
|    DF|  371|
|    PR|  370|
|    AP|  371|
+------+-----+



In [187]:
#Quantos CPFs válidos e inválidos foram encontrados?

df.groupBy('cpf_status').count().show()

+----------+-----+
|cpf_status|count|
+----------+-----+
|Não Valido|    1|
|    Valido| 9999|
+----------+-----+



In [188]:
#Quantos CNPJs válidos e inválidos foram encontrados?

df.groupBy('cnpj_status').count().show()

+-----------+-----+
|cnpj_status|count|
+-----------+-----+
|     Valido|10000|
+-----------+-----+



In [189]:
#Escrevendo o arquivo csv

df.write.option("header",True).mode("overwrite").csv("problema1_normalizado.csv")

In [190]:
#Escrevendo o arquivo parquet

df.write.option("header",True).mode("overwrite").parquet("problema1_normalizado.parquet")